# GEX Calculation using Yahoo Finance Data
Main thing to keep in mind is that YF data isn't as clean as CBOE. This is just to demonstrate how to use free resources to get what we need. While YF does not have ✨all✨ option data, it may still be useful for near-dated options. 

Uncomment below for requirements and pip install 👇🏽

In [1]:
# !pip install pandas
# !pip install numpy
# !pip install opstrat
# !pip install plotly
# !pip install yfinance
# !pip install tqdm
# !pip install json

In [2]:
import pandas_datareader as pdr
import opstrat as op
import datetime
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import yfinance as yf
from itertools import *
from tqdm import tqdm
import json

## Main Prices and Helper Functions
- `sofr`: SOFR rate from FRED as basis for risk-free rate.
- `spx`: For getting options data. Known issue with this is that the data from here does not align 100% with what's seen on CBOE. Not sure whether this can or will get fixed, but. 
- `gspc`: For getting SPX historical prices. Not sure why Yahoo has them under different tickers but 🤷🏽‍♂️
- `rmp`: For getting the current spot price of SPX
- `price`: Backup value for when `rmp` is not available. Happens sometimes.

In [3]:

sofr = pdr.get_data_fred('SOFR')
spx = yf.Ticker('^SPX')
gspc = yf.Ticker('^GSPC')

rmp = gspc.info['regularMarketPrice']
price = gspc.history(period='1mo',interval='1d')

high_1d = price.iloc[-1]['High']
low_1d = price.iloc[-1]['Low']
close_1d = price.iloc[-1]['Close']

# high_1w = spx_weekly.loc[today_dt, 'High']
# low_1w = spx_weekly.loc[today_dt, 'Low']
# close_1w = spx_weekly.loc[today_dt, 'Close']

def make_pivots(h, l, c):
    '''
    Helper function to calculate Classic pivot points. Only for reference.
    '''
    piv = (h + l + c) / 3
    pivRange = h - l
    pivR1 = piv * 2 - l
    pivS1 = piv * 2 - h
    pivR2 = piv + 1 * pivRange
    pivS2 = piv - 1 * pivRange
    pivR3 = piv + 2 * pivRange
    pivS3 = piv - 2 * pivRange
    pivR4 = piv + 3 * pivRange
    pivS4 = piv - 3 * pivRange
    return piv, pivR1, pivS1, pivR2, pivS2, pivR3, pivS3, pivR4, pivS4

piv, pivR1, pivS1, pivR2, pivS2, pivR3, pivS3, pivR4, pivS4 = make_pivots(high_1d, low_1d, close_1d)


def get_current_price():
    '''
    Returns the current price. Sometimes Yahoo will return nothing using .get_info, so this is a backup to get the current price.
    '''
    if rmp:
        return rmp
    elif not rmp:
        return price.iloc[-1]['Close']

# today = datetime.now() if today is None or today == '' else datetime.strptime(today,date_format)
current_price = float(get_current_price())

def calc_greeks(K=None, St=None, r=sofr.iloc[-1][0], t=None, v=None, type='c'):
    '''
    Yahoo Finance does not provide greeks, so they're calculated here.
    For each strike, given current stock price,
    calculate the greeks and return them
    as a dataframe.

    K=200    #spot price
    St=208   #current stock price
    r=sofr.iloc[-1][0]     #4% risk free rate
    t=30     #time to expiry, 30 days 
    v=20     #volatility 
    type='c' #Option type call
    '''
    bsm = op.black_scholes(K=K, St=St, r=r, t=t, v=v, type=type)
    return bsm

def getExpiry(contractSymbol):
    '''
    Helper function to get Expiry dates. Important distinction between weeklies and AMs, as the contract is slightly different.
    '''
    expiry = None
    if contractSymbol[0:4] == 'SPXW':
        expiry = contractSymbol[4:10]
    else:
        expiry = contractSymbol[3:9]
    return expiry

def zero_gex(strikes):
    '''
    Function to calculate Zero GEX level. Lifted this off a medium article somewhere. 
    '''
    def add(a, b):
        return (b[0], a[1] + b[1])

    cumsum = list(accumulate(strikes, add))
    if cumsum[len(strikes) // 10][1] < 0:
        op = min
    else:
        op = max
    return op(cumsum, key=lambda i: i[1])[0]


# Code for gathering options data
- Given dates, get option data from yfinance.
- Given option data, do an outer merge to see all available chain.
- KNOWN ISSUE: Some chains are just missing from Yahoo Finance, so CBOE may still be better. WIP is process to determine missing cons and getting them individually, but that code isn't ready yet.

In [4]:

all_calls = []
all_puts = []

dates = spx.options

# Outer join
# Calculate Greeks

# For each date, get the option chain
for date in tqdm(dates):

    oc = spx.option_chain(date=date)
    dte_days = ((datetime.datetime.strptime(date, '%Y-%m-%d') - datetime.datetime.now()).days) + 1
    dte_sec = (datetime.datetime.strptime(date, '%Y-%m-%d') - datetime.datetime.now()).seconds
    dte = (dte_days * 24 + dte_sec // 3600) / 24
    

    # Append to lists with intention to merge on expiry, strike and dte

    for opt, lst in zip([oc.calls, oc.puts], [all_calls, all_puts]):
        opt['dte'] = dte
        opt['ExpirationDate'] = datetime.datetime.strptime(date, '%Y-%m-%d')
        lst.append(opt)
    
    df_all_calls = pd.concat(all_calls)
    df_all_calls['Expiry'] = df_all_calls['contractSymbol'].apply(getExpiry)
    df_all_calls['Key1'] = df_all_calls['contractSymbol'].str[:4]

    df_all_puts = pd.concat(all_puts)
    df_all_puts['Expiry'] = df_all_puts['contractSymbol'].apply(getExpiry)
    df_all_puts['Key1'] = df_all_puts['contractSymbol'].str[:4]


    df_all_opts = df_all_calls.merge(df_all_puts, how = 'outer', on = ['strike','Expiry','Key1'])
    df_all_opts['DaysExp'] = df_all_opts[['dte_x','dte_y']].max(axis=1)
    df_all_opts['ExpirationDate'] = df_all_opts[['ExpirationDate_x','ExpirationDate_y']].max(axis=1)

# Impute values for implied impliedVolatility
df_all_opts['impliedVolatility_x'].fillna(0, inplace=True)
df_all_opts['impliedVolatility_y'].fillna(0, inplace=True)

df_all_opts['openInterest_x'].fillna(0, inplace=True)
df_all_opts['openInterest_y'].fillna(0, inplace=True)


100%|██████████| 47/47 [00:09<00:00,  4.91it/s]


# Code for calculating Greeks
- Use `opstrat` to calculate. Again, because there are missing chains from Yahoo Finance, it's not perfect.

In [5]:
# Iterate through calls and puts and compute greeks
# Append to a list for merging later

call_greeks = []
put_greeks = []

for _typ, vol, lst in zip(['c','p'], ['impliedVolatility_x','impliedVolatility_y'], [call_greeks, put_greeks]):
    for num, opt in enumerate(df_all_opts.values):
        _row = df_all_opts.iloc[num]
        _greeks = calc_greeks(
                K=_row['strike'], 
                St=current_price, 
                r=sofr.iloc[-1][0], 
                t=_row['DaysExp'], 
                v=_row[vol] * 100, 
                type=_typ
            )['greeks']
        lst.append(_greeks)

df_c_greeks = pd.DataFrame(call_greeks)
df_p_greeks = pd.DataFrame(put_greeks)

df_c_greeks = df_c_greeks.rename(columns={
    'delta':'CallDelta',
    'gamma':'CallGamma'
})

df_p_greeks = df_p_greeks.rename(columns={
    'delta':'PutDelta',
    'gamma':'PutGamma'
})

df_final = pd.concat([df_all_opts, df_c_greeks, df_p_greeks], axis=1)

df_final = df_final.rename(columns={
    'ExpirationDate': 'ExpirationDate',
    'contractSymbol_x': 'Calls',
    'lastPrice_x': 'CallLastSale',
    #CallNet: 'CallNet',
    'bid_x': 'CallBid',
    'ask_x': 'CallAsk',
    'volume_x': 'CallVol',
    'impliedVolatility_x': 'CallIV',
    'CallDelta': 'CallDelta',
    'CallGamma': 'CallGamma',
    'openInterest_x': 'CallOpenInt',
    'strike': 'CallStrike',
    'contractSymbol_y': 'Puts',
    'lastPrice_y': 'PutLastSale',
    #PutNet: 'PutNet',
    'bid_y': 'PutBid',
    'ask_y': 'PutAsk',
    'volume_y': 'PutVol',
    'impliedVolatility_y': 'PutIV',
    'PutDelta': 'PutDelta',
    'PutGamma': 'PutGamma',
    'openInterest_y': 'PutOpenInt'
})

today = datetime.datetime.now()

# df_final['DaysExp'] = (pd.to_datetime(
#     df_final['ExpirationDate'].str[3:]) - today).dt.days
df_final['DaysExp'] = df_final['DaysExp'].clip(0, 120)
df_final = df_final.groupby(['CallStrike', 'DaysExp', 'ExpirationDate']).agg({
    'Calls': lambda x: x.iloc[-1],
    'CallLastSale': np.nanmax,
    # 'CallNet': np.nanmax,
    'CallBid': np.nanmax,
    'CallAsk': np.nanmax,
    'CallVol': np.nansum,
    'CallIV': np.nanmean,
    'CallDelta': np.nanmean,
    'CallGamma': np.nanmean,
    'CallOpenInt': np.nansum,
    # 'CallStrike':np.max,
    'Puts': lambda x: x.iloc[-1],
    'PutLastSale': np.nanmax,
    # 'PutNet': np.nanmax,
    'PutBid': np.nanmax,
    'PutAsk': np.nanmax,
    'PutVol': np.nansum,
    'PutIV': np.nanmean,
    'PutDelta': np.nanmean,
    'PutGamma': np.nanmean,
    'PutOpenInt': np.nansum
}).reset_index()

df_final['CallOpenInt'] = df_final['CallOpenInt'].astype(float) + df_final['CallVol'].astype(float)
df_final['CallGEX'] = df_final['CallGamma'] * df_final['CallOpenInt'] * \
    100 * current_price  # (spot_price ** 2) * 0.01

df_final['CallGEX'] = df_final['CallGEX'].fillna(0)

df_final['PutOpenInt'] = df_final['PutOpenInt'].astype(float) + df_final['PutVol'].astype(float)
df_final['PutGEX'] = df_final['PutGamma'] * df_final['PutOpenInt'] * \
    100 * -1 * current_price  # (spot_price ** 2) * 0.01

df_final['PutGEX'] = df_final['PutGEX'].fillna(0)

df_final['TotalGamma'] = df_final['CallGEX'] + df_final['PutGEX']

total_gamma = df_final['TotalGamma'].sum()

count = 0
strikeWithGamma = []
for a, b in zip(df_final.CallStrike, df_final.TotalGamma):
    strikesPlusGamma = (a, b)
    strikeWithGamma.append(strikesPlusGamma)
df_final['StrikeAndGamma'] = strikeWithGamma

# new = new.sort_values('DaysExp', ascending=False)

# Code for Generating Chart
- Use `plotly` to visualize net GEX per strike, and generate a HTML file containing the chart.

In [6]:
new = df_final.loc[
    (df_final['TotalGamma'] != 0.00) &
    (~df_final['TotalGamma'].isnull())
    # ((new['CallOpenInt'] >= (new['CallOpenInt'].mean())) |
    #  (new['PutOpenInt'] >= (new['PutOpenInt'].mean())))
]

zero_strike = zero_gex(new.StrikeAndGamma)

long_df = new.sort_values(['CallStrike', 'DaysExp']).groupby(
    'CallStrike')['TotalGamma'].sum().reset_index()
long_df['TotalGammaPos'] = [
    x if x > 0 else np.nan for x in long_df['TotalGamma']]
long_df['TotalGammaNeg'] = [
    x if x < 0 else np.nan for x in long_df['TotalGamma']]


f = open('gex_settings.json')

ticker_dict = json.load(f)

# long_df.loc[long_df['TotalGamma'] < 0, 'GEXDirection'] = 'Negative'
total_gamma = new['TotalGamma'].sum()
long_title = f"""
Gamma Exposure for SPX/W
<br>Net GEX: {total_gamma/1000000:,.3f}M
"""
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=long_df['TotalGammaPos'],
        y=long_df['CallStrike'],
        orientation='h',
        name='Positive GEX',
        marker_color='#3399ff',
        # yperiodalignment="middle"
        width=3 # ticker_dict[ticker]["width"]
    )
)

fig.add_trace(
    go.Bar(
        x=long_df['TotalGammaNeg'],
        y=long_df['CallStrike'],
        orientation='h',
        name='Negative GEX',
        marker_color='#ff5f5f',
        # yperiodalignment="middle"
        width=3 # ticker_dict[ticker]["width"]
    )
)

fig.add_hline(y=current_price, line_width=2, line_dash="solid", line_color="#33ccb3",
              annotation_text=f"Last Price: {current_price:.2f}", annotation_position="top right")

fig.add_hline(y=zero_strike, line_width=2, line_dash="solid", line_color="#cc6633",
              annotation_text=f"Zero Strike: {zero_strike:.2f}", annotation_position="bottom right")

# Pivot lines

show_piv = True

if show_piv:
    fig.add_hline(y=piv, line_width=2, line_dash="solid", line_color="#9598a1",
                annotation_text=f"Piv: {piv:.2f}", annotation_position="top left", annotation_font_color="#9598a1", annotation_font_size=11, opacity=0.8)
    fig.add_hline(y=pivR1, line_width=2, line_dash="solid", line_color="#9598a1",
                annotation_text=f"R1: {pivR1:.2f}", annotation_position="top left", annotation_font_color="#9598a1", annotation_font_size=11, opacity=0.8)
    fig.add_hline(y=pivR2, line_width=2, line_dash="solid", line_color="#9598a1",
                annotation_text=f"R2: {pivR2:.2f}", annotation_position="top left", annotation_font_color="#9598a1", annotation_font_size=11, opacity=0.8)
    fig.add_hline(y=pivR3, line_width=2, line_dash="solid", line_color="#9598a1",
                annotation_text=f"R3: {pivR3:.2f}", annotation_position="top left", annotation_font_color="#9598a1", annotation_font_size=11, opacity=0.8)
    fig.add_hline(y=pivR4, line_width=2, line_dash="solid", line_color="#9598a1",
                annotation_text=f"R4: {pivR4:.2f}", annotation_position="top left", annotation_font_color="#9598a1", annotation_font_size=11, opacity=0.8)
    fig.add_hline(y=pivS1, line_width=2, line_dash="solid", line_color="#9598a1",
                annotation_text=f"S1: {pivS1:.2f}", annotation_position="bottom left", annotation_font_color="#9598a1", annotation_font_size=11, opacity=0.8)
    fig.add_hline(y=pivS2, line_width=2, line_dash="solid", line_color="#9598a1",
                annotation_text=f"S2: {pivS2:.2f}", annotation_position="bottom left", annotation_font_color="#9598a1", annotation_font_size=11, opacity=0.8)
    fig.add_hline(y=pivS3, line_width=2, line_dash="solid", line_color="#9598a1",
                annotation_text=f"S3: {pivS3:.2f}", annotation_position="bottom left", annotation_font_color="#9598a1", annotation_font_size=11, opacity=0.8)
    fig.add_hline(y=pivS4, line_width=2, line_dash="solid", line_color="#9598a1",
                annotation_text=f"S4: {pivS4:.2f}", annotation_position="bottom left", annotation_font_color="#9598a1", annotation_font_size=11, opacity=0.8)

fig.update_layout(
    title_text=long_title,
    font={
        'color': '#ffffff',
        'family': 'Roboto'
    },
    autosize=True,
    yaxis_range=[round(current_price * 0.95, 1), round(current_price * 1.05, 1)],
    yaxis_dtick=10, #ticker_dict[ticker]['yaxis_dtick'],
    plot_bgcolor="black",
    paper_bgcolor="black"
)
fig.update_traces(marker_line_width=0)
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#131722')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#131722')
# fig.show()
fig.write_html('net_gex.html', auto_open=True)